# 任务 2：部署用于实时推理的模型

## 任务 2.1：环境设置

安装程序包和依赖项。

In [ ]:
#install-dependencies
import boto3
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

查看经过处理的客户数据集。

In [ ]:
#explore-dataset
column_list = ['income','age','workclass','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.dtypes
lab_test_data.head()

将来自训练和优化实验的模型保存在默认 Amazon Simple Storage Service (Amazon S3) 存储桶中。使用 **create_model** 设置模型，并配置 **ModelDataUrl** 以引用经过训练的模型。

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## 任务 2.2：从提供的经过合成和重新训练的模型创建终端节点

使用 Amazon SageMaker 适用于 Python 的 SDK 创建终端节点分三个步骤。
1.创建 SageMaker 模型。
2.为 HTTPS 终端节点创建终端节点配置。
3.创建 HTTPS 终端节点。

有关创建终端节点的更多信息，请参阅 [Create Your Endpoint and Deploy Your Model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)。

您已创建模型。现在，您可以创建终端节点配置和终端节点。

首先，设置终端节点配置名称和要使用的实例类型。然后，调用 CreateEndpointConfig API。

要创建终端节点配置，您需要设置以下选项：
- **VariantName**：生产变体（生产环境中的一个或多个模型）的名称。
- **ModelName**：您要托管的模型的名称。这是您在创建模型时指定的名称。
- **InstanceType**：计算实例类型。
- **InitialInstanceCount**：最初要启动的实例数。

要将输入记录到您的终端节点以及将来自 SageMaker 实时终端节点的推理输出记录到 Amazon S3，您可以启用名为“数据捕获”的功能。数据捕获功能通常用于记录可用于训练、调试和监控的信息。在 Amazon SageMaker Studio 中浏览终端节点时，如果您启用数据捕获功能，将会看到有关终端节点的更多详细信息。本实验后面的数据捕获功能配置将展示如何启用该功能。

有关添加数据捕获功能的更多信息，请参阅 [Capture Data](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html)。

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date  
# so it you can search endpoints based on creation time.
endpoint_config_name = 'income-model-real-time-endpoint-{}'.format(create_date)                              
instance_type = 'ml.m5.xlarge'   
initial_sampling_percentage = 25 # Choose a value between 0 and 100
capture_modes = [ "Input",  "Output" ] # Specify input, output, or both

endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ],
    DataCaptureConfig= {
        'EnableCapture': True, # Whether data should be captured or not.
        'InitialSamplingPercentage' : initial_sampling_percentage,
        'DestinationS3Uri': f's3://{bucket}/data-capture',
        'CaptureOptions': [{"CaptureMode" : capture_mode} for capture_mode in capture_modes]
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

接下来，创建终端节点。当您创建实时终端节点时，SageMaker 会启动机器学习 (ML) 计算实例，并按照配置中指定的方式部署一个或多个模型。在本实验中，您将仅部署一个用于推理的模型。在 SageMaker 中，您可以创建多模型终端节点。有关多模型终端节点的更多信息，请参阅 [Invoke a Multi-Model Endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/invoke-multi-model-endpoint.html)。

当终端节点正在运行时，辅助函数将打印终端节点 Amazon Resource Name (ARN)。运行终端节点创建流程大约需要 3–7 分钟。

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


在 SageMaker Studio 中，您可以在 **Endpoints**（终端节点）选项卡下查看终端节点详细信息。

下一个步骤将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **real_time_inference.ipynb** 选项卡拖到一边，或者选择 **real_time_inference.ipynb** 选项卡，然后从工具栏中选择 **File**（文件）和 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览终端节点时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完终端节点后，通过选择 **real_time_inference.ipynb** 选项卡返回至笔记本。

1.选择 **SageMaker Home**（SageMaker 主页）图标。
2.选择 **Deployments**（部署）。
3.选择 **Endpoints**（终端节点）。

此时 SageMaker Studio 将显示 **Endpoints**（终端节点）选项卡。

4.选择 **Name**（名称）列中包含 **income-model-real-time-** 的终端节点。

如果该终端节点未出现，请选择刷新图标，直到该终端节点出现在列表中。

此时 SageMaker Studio 将显示 **ENDPOINT DETAILS**（终端节点详细信息）选项卡。

5.选择 **AWS settings**（AWS 设置）选项卡。

如果您在终端节点完成创建之前打开了终端节点，请选择刷新图标，直到 **Endpoint status**（终端节点状态）从 *Creating*（正在创建）变为 *InService*（正在运行）。

**Endpoint type**（终端节点类型）显示为 **Real-time**（实时）。**Data capture settings**（数据捕获设置）和 **Endpoint configuration settings**（终端节点配置设置）部分显示您之前在笔记本中选择的配置。

## 任务 2.3：使用实时客户记录调用终端节点以进行实时推理

使用 SageMaker 托管服务部署模型后，您可以向终端节点发送测试数据，以便在其上测试模型。

您知道有几条客户记录的收入大于或等于 50,000 USD（**income**（收入）值为 **1**），还有几条客户记录的收入低于 50,000 USD（**income**（收入）值为 **0**）。请使用这些记录调用终端节点，并查看返回的分数。

要查看来自终端节点的实时预测，请从响应中读取返回的正文文本，其中包含预测分数的列表。每条记录的分数介于 **0** 到 **1** 之间，数字接近 **1** 表示这些客户的收入更有可能大于或等于 50,000 USD。例如，预测分数为 **0.42** 的客户比预测分数为 **0.14** 的客户更有可能获得大于或等于 50,000 USD 的收入。

In [ ]:
#invoke-endpoint-real-time-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('56,3,6,6,0,3,1,0,0,1,0,13\n' +
                '29,2,2,2,0,1,0,0,0,0,0,70\n' +
                '79,0,1,1,0,3,5,0,0,0,0,20\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,3,2,0,0,0,0,32\n' +
                '31,0,1,1,2,1,2,1,1,0,0,40\n' +
                '23,0,1,1,1,0,1,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## 任务 2.4：删除终端节点

清理终端节点可以分三个步骤完成。首先，删除终端节点。然后，删除终端节点配置。最后，如果不再需要部署的模型，则删除该模型。

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### 总结

恭喜！ 您已通过 SageMaker 使用 SageMaker Python SDK 成功创建一个实时终端节点，并调用了该终端节点。

本实验的下一个任务侧重于部署使用无服务器推理进行推理的模型。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续执行**任务 3：部署用于无服务器推理的模型**。